---
# Estimating $\pi$ Using Monte Carlo Methods
--- 

## Introduction

$\pi$ is one of the most famous mathematical constants, representing the ratio of a circle's circumference to its diameter. This fundamental constant appears throughout mathematics and physics, and its value has fascinated mathematicians for centuries. While $\pi$  has been calculated to billions of decimal places using sophisticated algorithms, there's an elegant and intuitive way to estimate its value using random numbers: the **Monte Carlo method**.

## The Monte Carlo Approach

The Monte Carlo method is a powerful computational technique that uses random sampling to solve complex problems. For estimating π, this method leverages a beautiful geometric relationship between a circle and a square.

### The Core Idea

Imagine you have a square with side length 2, centered at the origin, so it extends from -1 to +1 in both x and y directions. Inside this square, inscribe a unit circle (radius = 1). Now, if you randomly scatter points throughout the square, some will fall inside the circle and others outside.

The key insight is that the **probability of a random point falling inside the circle is proportional to the ratio of the circle's area to the square's area**:

- Area of the unit circle: $\pi \times 1^2 = \pi$ 
- Area of the square: $2^2 = 4$
- Probability ratio: $\pi$/

Therefore, if we generate many random points and count how many fall inside the circle, we can estimate π!

### The Algorithm

The Monte Carlo estimation of π follows these simple steps:

1. **Generate random points**: Create random coordinates $(x, y)$ within the square $[-1, 1] \times [-1, 1]$
2. **Test circle membership**: For each point, check if $x^2 + y^2 \leq 1$ (inside the unit circle)
3. **Count and repeat**: Keep track of points inside vs. total points generated
4. **Estimate $\pi$**: Calculate the ratio of inside points to total points, then multiply by 4

**Formula**: $\pi \sim  4 \times$  (number of points inside circle) / (total number of points)

### Why This Works

The Monte Carlo method works because of the **Law of Large Numbers**. As we increase the number of random samples, our estimate converges toward the true value of π. The relationship between sample size, estimation accuracy, and error provides fascinating insights into both probability theory and computational mathematics.

In this notebook, we'll implement this algorithm, visualize the random sampling process, and explore how the estimate improves with sample size through stunning visualizations that demonstrate the power of statistical sampling.

---


In [2]:
import random

def estimate_pi(num_samples: int) -> float:
    """
    Estimate the value of π using a Monte Carlo simulation.

    The method works by randomly generating points (x, y) in the square
    [-1, 1] × [-1, 1] and counting how many fall inside the unit circle
    defined by x² + y² ≤ 1. The ratio of points inside the circle to the
    total number of samples approximates π/4.

    Args:
        num_samples (int): Number of random samples to generate.

    Returns:
        float: Estimated value of π.
    """
    num_inside = 0  # Counter for points inside the circle

    for _ in range(num_samples):
        # Generate random point in the square [-1, 1] × [-1, 1]
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        # Check if the point lies inside the unit circle
        if x**2 + y**2 <= 1:
            num_inside += 1

    # π is approximately 4 × (points inside / total points)
    return 4 * num_inside / num_samples


if __name__ == "__main__":
    # Run the simulation with 10,000 samples
    pi_estimate = estimate_pi(10_000)
    print(f"Estimated value of π (10,000 samples): {pi_estimate:.6f}")

Estimated value of π (10,000 samples): 3.150000


In [4]:
import numpy as np
import plotly.graph_objects as go

def monte_carlo_pi(num_samples: int, step: int = 500):
    """
    Monte Carlo simulation of π 

    Args:
        num_samples (int): Total number of random samples.
        step (int): Number of points to add per animation frame.

    Returns:
        (x, y, inside, estimates): Arrays of coordinates, boolean mask for inside points,
                                   and progressive π estimates.
    """
    # Generate all points
    x = np.random.uniform(-1, 1, num_samples)
    y = np.random.uniform(-1, 1, num_samples)

    # Inside-circle mask
    inside = x**2 + y**2 <= 1

    # Progressive estimates
    estimates = []
    for n in range(step, num_samples + 1, step):
        inside_n = np.sum(inside[:n])
        estimates.append(4 * inside_n / n)

    return x, y, inside, estimates, step


if __name__ == "__main__":
    num_samples = 10_000
    step = 500  # points per animation frame

    x, y, inside, estimates, step = monte_carlo_pi(num_samples, step)

    # Base figure
    fig = go.Figure()

    # Initial scatter (just first batch of points)
    fig.add_trace(go.Scatter(
        x=x[:step][inside[:step]], y=y[:step][inside[:step]],
        mode="markers", name="Inside Circle",
        marker=dict(color="blue", size=4, opacity=0.6)
    ))
    fig.add_trace(go.Scatter(
        x=x[:step][~inside[:step]], y=y[:step][~inside[:step]],
        mode="markers", name="Outside Circle",
        marker=dict(color="gray", size=4, opacity=0.6)
    ))

    # Circle boundary
    theta = np.linspace(0, 2*np.pi, 300)
    fig.add_trace(go.Scatter(
        x=np.cos(theta), y=np.sin(theta),
        mode="lines", name="Unit Circle",
        line=dict(color="red", width=2)
    ))

    # Frames for animation
    frames = []
    for k, estimate in enumerate(estimates, start=1):
        n = k * step
        frame = go.Frame(
            data=[
                go.Scatter(x=x[:n][inside[:n]], y=y[:n][inside[:n]]),
                go.Scatter(x=x[:n][~inside[:n]], y=y[:n][~inside[:n]])
            ],
            name=str(k),
            layout=go.Layout(
                title=f"Monte Carlo Estimate of π ≈ {estimate:.4f} (N={n})"
            )
        )
        frames.append(frame)

    # Animation settings
    fig.update(frames=frames)
    fig.update_layout(
        title=f"Monte Carlo Estimate of π (N={step})",
        xaxis=dict(scaleanchor="y", range=[-1, 1], title="x"),
        yaxis=dict(range=[-1, 1], title="y"),
        width=650, height=650,
        updatemenus=[dict(
            type="buttons",
            showactive=False,
            buttons=[
                dict(label="▶ Play", method="animate",
                     args=[None, {"frame": {"duration": 200, "redraw": True},
                                  "fromcurrent": True, "mode": "immediate"}]),
                dict(label="⏸ Pause", method="animate",
                     args=[[None], {"frame": {"duration": 0, "redraw": False},
                                    "mode": "immediate"}])
            ]
        )]
    )

    fig.show()

## Convergence of Estimate

In [5]:
import random
import numpy as np
import plotly.graph_objects as go

# Monte Carlo π estimation
def estimate_pi(n):
    count = 0
    for i in range(n):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            count += 1
    return 4 * count / n

# Sample sizes (log spaced)
sample_sizes = np.logspace(1, 6, num=40, dtype=int)

# Progressive π estimates
pi_estimates = [estimate_pi(n) for n in sample_sizes]

# Base figure
fig = go.Figure()

# Initial trace
fig.add_trace(go.Scatter(
    x=[sample_sizes[0]], y=[pi_estimates[0]],
    mode="markers+lines", name="π Estimate",
    line=dict(color="blue"), marker=dict(size=8)
))

# True π line
fig.add_trace(go.Scatter(
    x=[sample_sizes[0], sample_sizes[-1]], 
    y=[np.pi, np.pi],
    mode="lines", name="True π",
    line=dict(color="red", dash="dash")
))

# Frames for animation
frames = []
for k in range(1, len(sample_sizes)):
    frame = go.Frame(
        data=[
            go.Scatter(x=sample_sizes[:k+1], y=pi_estimates[:k+1]),
            go.Scatter(x=[sample_sizes[0], sample_sizes[-1]], y=[np.pi, np.pi])
        ],
        name=str(k),
        layout=go.Layout(
            title=f"Monte Carlo π Estimate (N={sample_sizes[k]:,}) ≈ {pi_estimates[k]:.5f}"
        )
    )
    frames.append(frame)

# Layout & animation controls
fig.update(frames=frames)
fig.update_layout(
    title="Monte Carlo π Convergence",
    xaxis=dict(title="Number of Points (log scale)", type="log"),
    yaxis=dict(title="Estimated π Value", range=[2.5, 3.8]),
    width=700, height=500,
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[
            dict(label="▶ Play", method="animate",
                 args=[None, {"frame": {"duration": 300, "redraw": True},
                              "fromcurrent": True, "mode": "immediate"}]),
            dict(label="⏸ Pause", method="animate",
                 args=[[None], {"frame": {"duration": 0, "redraw": False},
                                "mode": "immediate"}])
        ]
    )]
)

fig.show()

## Error vs. Sample Size

In [6]:
import random
import numpy as np
import plotly.graph_objects as go

# Function to estimate pi using Monte Carlo
def estimate_pi(n):
    count = 0
    for _ in range(n):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            count += 1
    return 4 * count / n

# Sample sizes (log spaced)
sample_sizes = np.logspace(1, 6, num=40, dtype=int)

# Estimates and errors
pi_estimates = np.array([estimate_pi(n) for n in sample_sizes])
pi_errors = np.abs(pi_estimates - np.pi)

# Base figure
fig = go.Figure()

# Initial trace
fig.add_trace(go.Scatter(
    x=[sample_sizes[0]], y=[pi_errors[0]],
    mode="markers+lines", name="Error",
    line=dict(color="black"), marker=dict(size=8)
))

# Frames for animation
frames = []
for k in range(1, len(sample_sizes)):
    frame = go.Frame(
        data=[
            go.Scatter(x=sample_sizes[:k+1], y=pi_errors[:k+1]),
        ],
        name=str(k),
        layout=go.Layout(
            title=f"Error in π Estimate (N={sample_sizes[k]:,}) → Error={pi_errors[k]:.2e}"
        )
    )
    frames.append(frame)

# Layout & controls
fig.update(frames=frames)
fig.update_layout(
    title="Monte Carlo π Estimation Error",
    xaxis=dict(title="Sample Size (log scale)", type="log"),
    yaxis=dict(title="Absolute Error (log scale)", type="log"),
    width=700, height=500,
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[
            dict(label="▶ Play", method="animate",
                 args=[None, {"frame": {"duration": 300, "redraw": True},
                              "fromcurrent": True, "mode": "immediate"}]),
            dict(label="⏸ Pause", method="animate",
                 args=[[None], {"frame": {"duration": 0, "redraw": False},
                                "mode": "immediate"}])
        ]
    )]
)

fig.show()